In [1]:
import pandas as pd # DataFrame library
import os # Operating system library

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./secrets/my_credentials.json" # Credentials to be able to access the Google BigQuery public datasets

In [3]:
raw_weather_query = """SELECT weather.id, weather.date, weather.time, weather.element, weather.value, weather.qflag,
                       stations.latitude, stations.longitude, stations.name, stations.elevation 
                       FROM `bigquery-public-data.ghcn_d.ghcnd_*` AS weather
                       INNER JOIN `bigquery-public-data.ghcn_d.ghcnd_stations` AS stations
                       ON weather.id = stations.id
                       WHERE weather.id LIKE 'US%'
                       AND EXTRACT(YEAR FROM weather.date) BETWEEN 1990 AND 2023
                       AND weather._TABLE_SUFFIX BETWEEN '1990' AND '2023'
                       AND stations.id LIKE 'US%'
                       AND stations.state = 'IL'; """ # This query extracts weather data from the US in Illinois from the year 1990 to 2023
raw_weather_data = pd.read_gbq(raw_weather_query, use_bqstorage_api = True, progress_bar_type = "tqdm_notebook", dialect = "standard") # Load the data from the SQL query through Google BigQuery

Downloading:   0%|          |

In [4]:
raw_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19084993 entries, 0 to 19084992
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         object 
 1   date       dbdate 
 2   time       object 
 3   element    object 
 4   value      float64
 5   qflag      object 
 6   latitude   float64
 7   longitude  float64
 8   name       object 
 9   elevation  float64
dtypes: dbdate(1), float64(4), object(5)
memory usage: 1.4+ GB


In [5]:
raw_weather_data.shape

(19084993, 10)

In [6]:
desired_types = {"id": "string", 
                 "date": "string", 
                 "time": "string", 
                 "name": "string", 
                 "element": "string", 
                 "value": "float", 
                 "latitude": "float", 
                 "longitude": "float",
                 "elevation": "float",
                 "qflag": "string"}

raw_weather_data = raw_weather_data.astype(desired_types)
raw_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19084993 entries, 0 to 19084992
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         string 
 1   date       string 
 2   time       string 
 3   element    string 
 4   value      float64
 5   qflag      string 
 6   latitude   float64
 7   longitude  float64
 8   name       string 
 9   elevation  float64
dtypes: float64(4), string(6)
memory usage: 1.4 GB


In [7]:
raw_weather_data.to_parquet("./data/raw_weather.parquet", index = False)